In [10]:
import torch
from easydict import EasyDict
import numpy as np
from pathlib import Path
from sixdrepnet.model import SixDofNet, SixDRepNet

weights = './sixdrepnet/output/snapshots/resnet18_1681645840_bs80_112/last_checkpoint.tar'

model_path = Path(weights)
model_name = model_path.parts[-2].split('_')[0]
print(model_name)

model_checkpoint = torch.load(model_path)

if 'RepVGG' in model_name:
    model = SixDRepNet(backbone_name=model_name,
                                backbone_file='',
                                deploy=False,
                                pretrained=False)
else:
    model = SixDofNet(model_name, pretrained=False)

model.load_state_dict(model_checkpoint['model_state_dict'])

resnet18


<All keys matched successfully>

In [14]:
import torch
from easydict import EasyDict
import numpy as np
from pathlib import Path

def file_size(file):
    # Return file size in MB
    return Path(file).stat().st_size / 1e6


opt = EasyDict()
opt.weights = './sixdrepnet/output/snapshots/resnet18_1681645840_bs80_112/last_checkpoint.tar'
opt.output_name = 'resnet18_dof.onnx'
opt.img_size = 112
opt.dynamic = False
output_names = ['output']
opt.simplify = True
prefix = 'ONNX:'

img = torch.zeros([1,3,opt.img_size, opt.img_size], dtype=torch.float32)

try:
    import onnx

    print(f'{prefix} starting export with onnx {onnx.__version__}...')
    
    torch.onnx.export(model, img, opt.output_name, verbose=False, opset_version=11, input_names=['images'], output_names=output_names,
                            dynamic_axes=None)
    # Checks
    model_onnx = onnx.load(opt.output_name)  # load onnx model
    onnx.checker.check_model(model_onnx)  # check onnx model
    # print(onnx.helper.printable_graph(model_onnx.graph))  # print
    # Simplify
    if opt.simplify:
        try:
            import onnxsim

            print(f'{prefix} simplifying with onnx-simplifier {onnxsim.__version__}...')
            model_onnx, check = onnxsim.simplify(model_onnx,
                                                    dynamic_input_shape=opt.dynamic,
                                                    input_shapes={'images': list(img.shape)} if opt.dynamic else None)
            assert check, 'assert check failed'
            onnx.save(model_onnx, opt.output_name)
        except Exception as e:
            print(f'{prefix} simplifier failure: {e}')
    print(f'{prefix} export success, saved as {opt.output_name} ({file_size(opt.output_name):.1f} MB)')
except Exception as e:
    print(f'{prefix} export failure: {e}')



ONNX: starting export with onnx 1.12.0...
ONNX: simplifying with onnx-simplifier 0.4.19...
ONNX: export success, saved as resnet18_dof.onnx (44.7 MB)
